# Eaglesong

## Introduction

In this notebook, we will cover the essentials of building a telegram bots with `eaglesong` library. Telegram, as well as other messengers, offer the "per request" approach: the chatbot is given the incoming message and must produce the result. However, in case of longer conversations, it is easier to code the process from the chatbot's point of view: say this, listen to human, parse the input, say something else. `eaglesong` provides exactly this possibility. You write _chat flows_ like this:

```
yield "First question"
yield Listen()
first_answer = context.input

yield "Second question"
yield Listem()
second_question = context.input
```

and then control system builds a Telegram bot (or bot for other media) from such flows.

There are several onboarding scripts in this directory, each of them representing one chatbot. The code and comments for these chatbots are placed in this notebook fo convenience. Aside from reading, you can:

1. Run the scripts as a Telegram bot. To do that, you would need:
2. `eaglesong` provides a testbed environment to run chatbots in the notebook, see `Sandbox` notebook. This environment is not as well-studied as Telegram, so problems may appear when executing bots.
3. Read and run the tests. All the bots in the demos are tested with unit tests, and you can find them in `/kaia_tests/test_eaglesong/test_demo_skills`.

### Advanced notes

When a Telegram bot receives the very request for an update, it creates an iterator over
`main` function, and pulls commands from it until it's `Listen`. At this point the request
is considered complete, iterator is stored and the bot return the control to Telegram loop.
On the second request, it will restore the iterator and continue with the updated `context`
field from the exactly same point where it was interrupted.

Some people suggested `yield` approach can be replaced with async/await,
keeping the logic of the conversation flow intact.
Some other people, however, offered arguments why these approaches,
although similar and based on the same design pattern, are not equivalent in Python
and hence async/await cannot be used in this particular case.

* Unfortunately, my understanding of async/await does not allow me to answer this question with certainty.
    If someone wants to reimplement eaglesong with async/await, this and further demos provide
    a good understanding of the use cases that need to be considered.
* In general, I don't believe
    writing `await` instead of `yield` will improve anything. Althout, we could benefit
    from some standard await management from `asyncio` when it comes to complicated cases (with tasks).
* Both approaches should be able to coexist side-by-side with `Automaton` class abstraction.

## Setting things up

First we run the `eaglesong` unittest. If the following cell doesn't print `Success`, something if wrong and you need to fix it before proceeding.

In [1]:
import subprocess
import sys
from IPython.display import clear_output
assert 0 == subprocess.call([sys.executable, '-m', 'unittest', 'discover', '../../kaia_tests/test_eaglesong'])
clear_output()
print('Success!')

Success!


Before running the bots from `demos/eaglesong`, you will need:

1. Contact `@BotFather` bot on Telegram and register your chatbot. As the result, you will obtain an API key that looks like this: `0000000000:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx`
2. Create an `environment.env` file in the repository's root. This file is already in `.gitignore`, so don't be afraid to accidentaly push its contents.
3. Place `KAIA_TEST_BOT=<YOUR_API_KEY>` in the `environment.env` file.

The following cell checks if this was successful:

In [2]:
from kaia.infra import Loc
import os 

assert 'KAIA_TEST_BOT' in os.environ, 'Bot API KEY was not found'
print('Success!')

Success!


## Part 1. Simplest bots

Bots in this section will help you to understand how to design the chat-flow with eaglesong, and are sufficient to write a easy-to-medium chatbot yourself.

In [3]:
from IPython.display import HTML
from yo_fluq_ds import *
import markdown
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

def browse_files(files):
    html = []
    for file in files:
        html.append(f'<h3>{file.name}</h3>')
        text = FileIO.read_text(file)
        doc = []
        code = []
        in_doc = False
        for line in text.split('\n'):
            if line=='"""':
                in_doc = not in_doc
                continue
            if in_doc:
                doc.append(line)
            else:
                code.append(line)
        html.append(markdown.markdown('\n'.join(doc)))
        html.append('<br><br>')
        formatter = HtmlFormatter()
        python_code = highlight('\n'.join(code), PythonLexer(), formatter)       
        html.append(f'<style>{formatter.get_style_defs()}</style>{python_code}')
    return HTML(''.join(html))

browse_files(Query.folder('.').where(lambda z: z.name<='example_10' and 'example_01'<=z.name).order_by(lambda z: z.name).to_list())

## Part 2. Deep dive

This part will help you to understand how the chat flow you're writing is then translated to Telegram. Also, it will cover working with big asynchronous tasks (e.g., machine learning tasks) and how to integrate them in the flow.

**This part may be skipped entirely, if insides of `eaglesong` are not that interesting to you**. In the Part 3, we will show a user-friendly way to integrate machine-learning tasks in your code, that is no different from the flows in Part 1.

### Part 2.1. Automatons, Interpreters and Filters

The life cycle of the `Routine` is as follows:
* First, it is wrapped in various `Filters` that interpret some of `Routine`'s output. For instance, `PushdownFilter` processes the situation, when the output of `Routine` is another `Routine`: in this case, the routine is unpacked and executed as well.
* Then, the `Automaton` is created. Automaton creates an iterator over the subroutine, and runs this iterator by request. 
* Finally, there is an `Interpreter`. When receiving the request from the application, it executes the `Automaton` until specific conditions are met (usually, `Listen`, `Return` and `Terminate` are such conditions). Also, `Interpreter` executes the command that were not handled by the filters, in an application-specific way.

`TelegramTranslationFilter` is a filter that performs translation: everything that comes out of the flow, is transformed into `TgCommand` that is later executed by `TelegramInterpreter`. Also, the context that is obtained from Tegegram (and has a very detailed structure) is translated into simpler `BotContext` that we were using before.


In [4]:
browse_files(Query.folder('.').where(lambda z: z.name<='example_12' and 'example_10'<=z.name).order_by(lambda z: z.name).to_list())

### Part 2.2. Timers and Jobs

In [5]:
browse_files(Query.folder('.').where(lambda z: z.name<='example_16' and 'example_12'<=z.name).order_by(lambda z: z.name).to_list())

## Part 3. Narration

In this part, we introduce the way of seamless integration of ML tasks in the chatbot. This way comes from `kaia.narration` module.

The idea of narration is that we write essentially the same bots as before, but allow the ML-algorithms to improvise here and there, creating more lively experience. For that, we introduce `NarrationContext`, which inherits `BotContext`, but additionally has:

* `stage` field, containing the details about the customer, the character of the bot, and the history of past interactions. This will be useful to create a context for language models.
* `narrator()` function, that allow you to yield the commands like `narrator().improvise(text)` or `narrator().answer(text)`. These commands will be interpreted by `NarratorTranslationFilter`. 

Also, narration requires the special `Dispatcher` that allows everything to function. Currently, there is only `SimpleNarrationDispatcher`, that supports one skill. The question of how to integrate the multiple skills, especially when the can interrupt each other (for instance, a "Set Timer" skill may interrupt some less urgent skills) is non-trivial and it's better to cross this bridge when we face it.

In [6]:
browse_files(Query.folder('.').where(lambda z: z.name<='example_18' and 'example_16'<=z.name).order_by(lambda z: z.name).to_list())

## Part 4. Playground

In the following cell, you can run almost all the examples in the notebook and play with them. You can also 
develop your own chat bot in the `Sandbox` notebook.

For that, we should first start the TaskProcessor:

In [7]:
from kaia.infra.tasks import SqlSubprocTaskProcessor, SubprocessConfig

proc = SqlSubprocTaskProcessor(SubprocessConfig(
    task_class_path='kaia.infra.tasks:Waiting',
    task_args=[1], 
    db_path = Loc.temp_folder/'notebook_comm.db'
))
if not proc.is_alive(3):
    print('Process is not alive, activating')
    proc.activate()
    if not proc.is_alive(3):
        raise ValueError('Could not activate proc')
else:
    print('Process is alive')

Process is alive


Then, `search_bot` function will look for the example file that contains the search string, and extract `bot` from the found file. After that, we can run the widget that imitates bot:

In [8]:
from kaia.eaglesong.drivers.ipython import IPythonInterpreter, IPythonChatWidget, IPythonChatModel
import importlib

def get_bot_from_path(path):
    module_name = 'demos.eaglesong.'+path.name.split('.')[0]
    spec = importlib.util.spec_from_file_location(module_name, path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module.bot

def search_bot(search):
    path = Query.folder('.').where(lambda z: search in z.name).single()
    bot = get_bot_from_path(path)
    return bot

bot = search_bot('09')
bot.processor = proc
interpreter = IPythonInterpreter(bot.create_generic_routine(), IPythonChatModel())
chat = IPythonChatWidget(interpreter, timer=bot.timer, bot_name = bot.name)
chat.run()